In [60]:
import pandas as pd
import numpy as np

import requests
import io
from pypdf import PdfReader
pd.options.display.max_columns = None

In [108]:
df = pd.read_csv("data/individual_market_medical.csv")
df.head(2)

,Unnamed: 0,State Code,FIPS County Code,County Name,Metal Level,Issuer Name,HIOS Issuer ID,Plan ID (Standard Component),Plan Marketing Name,Standardized Plan Option,Plan Type,Rating Area,Child Only Offering,Source,Customer Service Phone Number Local,Customer Service Phone Number Toll Free,Customer Service Phone Number TTY,Network URL,Plan Brochure URL,Summary of Benefits URL,Drug Formulary URL,Adult Dental,Child Dental,EHB Percent of Total Premium,Premium Scenarios,Premium Child Age 0-14,Premium Child Age 18,Premium Adult Individual Age 21,Premium Adult Individual Age 27,Premium Adult Individual Age 30,Premium Adult Individual Age 40,Premium Adult Individual Age 50,Premium Adult Individual Age 60,Premium Couple 21,Premium Couple 30,Premium Couple 40,Premium Couple 50,Premium Couple 60,"Couple+1 child, Age 21","Couple+1 child, Age 30","Couple+1 child, Age 40","Couple+1 child, Age 50","Couple+2 children, Age 21","Couple+2 children, Age 30","Couple+2 children, Age 40","Couple+2 children, Age 50","Couple+3 or more Children, Age 21","Couple+3 or more Children, Age 30","Couple+3 or more Children, Age 40","Couple+3 or more Children, Age 50","Individual+1 child, Age 21","Individual+1 child, Age 30","Individual+1 child, Age 40","Individual+1 child, Age 50","Individual+2 children, Age 21","Individual+2 children, Age 30","Individual+2 children, Age 40","Individual+2 children, Age 50","Individual+3 or more children, Age 21","Individual+3 or more children, Age 30","Individual+3 or more children, Age 40","Individual+3 or more children, Age 50",Standard Plan Cost Sharing,Medical Deductible - Individual - Standard,Drug Deductible - Individual - Standard,Medical Deductible - Family - Standard,Drug Deductible - Family - Standard,Medical Deductible - Family (Per Person) - Standard,Drug Deductible - Family (Per Person) - Standard,Medical Maximum Out Of Pocket - Individual - Standard,Drug Maximum Out Of Pocket - Individual - Standard,Medical Maximum Out Of Pocket - Family - Standard,Drug Maximum Out Of Pocket - Family - Standard,Medical Maximum Out Of Pocket - Family (Per Person) - Standard,Drug Maximum Out Of Pocket - Family (Per Person) - Standard,Primary Care Physician - Standard,Specialist - Standard,Emergency Room - Standard,Inpatient Facility - Standard,Inpatient Physician - Standard,Generic Drugs - Standard,Preferred Brand Drugs - Standard,Non-preferred Brand Drugs - Standard,Specialty Drugs - Standard,73 Percent Actuarial Value Silver Plan Cost Sharing,Medical Deductible - Individual - 73 Percent,Drug Deductible - Individual - 73 Percent,Medical Deductible - Family - 73 Percent,Drug Deductible - Family - 73 Percent,Medical Deductible - Family (Per Person) - 73 Percent,Drug Deductible - Family (Per Person) - 73 Percent,Medical Maximum Out Of Pocket - Individual - 73 Percent,Drug Maximum Out Of Pocket - Individual - 73 Percent,Medical Maximum Out Of Pocket - Family - 73 Percent,Drug Maximum Out Of Pocket - Family - 73 Percent,Medical Maximum Out Of Pocket - Family (Per Person) - 73 Percent,Drug Maximum Out Of Pocket - Family (Per Person) - 73 Percent,Primary Care Physician - 73 Percent,Specialist - 73 Percent,Emergency Room - 73 Percent,Inpatient Facility - 73 Percent,Inpatient Physician - 73 Percent,Generic Drugs - 73 Percent,Preferred Brand Drugs - 73 Percent,Non-preferred Brand Drugs - 73 Percent,Specialty Drugs - 73 Percent,87 Percent Actuarial Value Silver Plan Cost Sharing,Medical Deductible - Individual - 87 Percent,Drug Deductible - Individual - 87 Percent,Medical Deductible - Family - 87 Percent,Drug Deductible - Family - 87 Percent,Medical Deductible - Family (Per Person) - 87 Percent,Drug Deductible - Family (Per Person) - 87 Percent,Medical Maximum Out Of Pocket - Individual - 87 Percent,Drug Maximum Out Of Pocket - Individual - 87 Percent,Medical Maximum Out Of Pocket - Family - 87 Percent,Drug Maximum Out Of Pocket - Family - 87 Percent,Medical Maximum Out Of Pocket - Family (Per Person) - 87 Percent,Drug Maximum Out Of Pocket 

In [74]:
print(df.columns)

Index(['Unnamed: 0', 'State Code', 'FIPS County Code', 'County Name',
       'Metal Level', 'Issuer Name', 'HIOS Issuer ID',
       'Plan ID (Standard Component)', 'Plan Marketing Name',
       'Standardized Plan Option',
       ...
       'Drug Maximum Out Of Pocket - Family (Per Person) - 94 Percent',
       'Primary Care Physician - 94 Percent', 'Specialist - 94 Percent',
       'Emergency Room - 94 Percent', 'Inpatient Facility - 94 Percent',
       'Inpatient Physician - 94 Percent', 'Generic Drugs - 94 Percent',
       'Preferred Brand Drugs - 94 Percent',
       'Non-preferred Brand Drugs - 94 Percent',
       'Specialty Drugs - 94 Percent'],
      dtype='object', length=150)


In [112]:
df.columns = df.columns.str.replace(r'Premium Adult Individual Age ', 'Prem Single ', regex=True)
df.columns = df.columns.str.replace(r'Premium Couple ', 'Prem Coup ', regex=True)
df.columns = df.columns.str.replace(r'Couple\+1 child, Age ', 'Coup1 ', regex=True)
df.columns = df.columns.str.replace(r'Couple\+2 children, Age ', 'Coup2 ', regex=True)

In [114]:
df_single = df.groupby(['State Code'])[['Prem Single 21', 'Prem Single 30 ', 'Prem Single 40 ', 'Prem Single 50 ', 'Prem Single 60 ']].median().reset_index()
df_single.head(2)

,State Code,Prem Single 21,Prem Single 30,Prem Single 40,Prem Single 50,Prem Single 60
0,AK,739.76,839.62,945.41,1321.20,2007.70
1,AL,438.10,497.23,559.87,782.42,1188.97


In [116]:
df_fam4 = df.groupby(['State Code'])[['Coup2 21', 'Coup2 30 ', 'Coup2 40 ', 'Coup2 50']].median().reset_index()
df_fam4.head(2)

,State Code,Coup2 21,Coup2 30,Coup2 40,Coup2 50
0,AK,2611.34,2811.06,3022.64,3774.22
1,AL,1432.56,1550.82,1676.10,2121.20


In [138]:
# join median income
state = pd.read_csv("data/states.csv")
income = pd.read_csv("data/med_income_2024.csv")
state_income = state.merge(income, how = "inner", on="State")
state_income = state_income[['Abbreviation', 'Income']]
state_income = state_income.rename(columns = {'Abbreviation':'State Code'})
state_income['Income'] = pd.to_numeric(state_income['Income'])
state_income.head(2)

,State Code,Income
0,AL,77363.0
1,AK,104081.0


In [140]:
df_fam4_inc = df_fam4.merge(state_income, how="inner", on = "State Code")
df_fam4_inc.head(2)

,State Code,Coup2 21,Coup2 30,Coup2 40,Coup2 50,Income
0,AK,2611.34,2811.06,3022.64,3774.22,104081.0
1,AL,1432.56,1550.82,1676.10,2121.20,77363.0


In [156]:
df_fam4_inc['yearly coup2 40'] = df_fam4_inc['Coup2 40 ']*12

df_fam4_inc['perc coup2 40'] = df_fam4_inc['yearly coup2 40'].div(df_fam4_inc['Income'])
df_fam4_inc.sort_values(by="perc coup2 40", ascending=False)

,State Code,Coup2 21,Coup2 30,Coup2 40,Coup2 50,Income,yearly coup2 40,perc coup2 40
29,WV,2640.22,2842.16,3056.06,3815.96,71678.0,36672.72,0.511631
0,AK,2611.34,2811.06,3022.64,3774.22,104081.0,36271.68,0.348495
30,WY,2296.02,2471.64,2657.66,3318.50,92028.0,31891.92,0.346546
11,LA,1624.12,1748.34,1879.94,2347.38,74911.0,22559.28,0.301148
18,NE,1903.18,2048.72,2202.92,2750.68,92614.0,26435.04,0.285432
5,FL,1674.29,1802.34,1938.00,2419.88,81514.0,23256.00,0.285301
16,NC,1697.32,1827.12,1964.64,2453.16,82890.0,23575.68,0.284421
15,MT,1698.72,1828.66,1966.28,2455.20,84027.0,23595.36,0.280807
24,SD,1794.70,1931.98,2077.38,2593.94,88996.0,24928.56,0.280109
14,MS,1332.52,1442.54,1559.08,1973.10,67731.0,18708.96,0.276224


In [147]:

#for each row, calculate the decade over decade increase in premiums 

percent_increase_single = df_single.pct_change(axis='columns')
percent_increase_single

,Prem Single 21,Prem Single 30,Prem Single 40,Prem Single 50,Prem Single 60
State Code,,,,,
AK,NaN,0.134990,0.125997,0.397489,0.519603
AL,NaN,0.134969,0.125978,0.397503,0.519606
AR,NaN,0.134999,0.125993,0.397493,0.519596
AZ,NaN,0.135020,0.125975,0.397501,0.519597
DE,NaN,0.134963,0.125984,0.397503,0.519599
FL,NaN,0.134987,0.126001,0.397486,0.519596
HI,NaN,0.135006,0.125986,0.397487,0.519610
IA,NaN,0.134970,0.125985,0.397490,0.519600
IL,NaN,0.135002,0.125994,0.397496,0.519586


In [159]:
df_couple = df.groupby(['State Code'])[['Prem Coup 30 ', 'Prem Coup 40 ', 'Prem Coup 50 ', 'Prem Coup 60 ']].median()

#for each row, calculate the decade over decade increase in premiums 

percent_increase_couple = df_couple.pct_change(axis='columns')
percent_increase_couple

,Prem Coup 30,Prem Coup 40,Prem Coup 50,Prem Coup 60
State Code,,,,
AK,NaN,0.125997,0.397489,0.519603
AL,NaN,0.125978,0.397503,0.519606
AR,NaN,0.125993,0.397493,0.519596
AZ,NaN,0.125975,0.397501,0.519597
DE,NaN,0.125984,0.397503,0.519599
FL,NaN,0.126001,0.397486,0.519596
HI,NaN,0.125986,0.397487,0.519610
IA,NaN,0.125985,0.397490,0.519600
IL,NaN,0.125994,0.397496,0.519586


In [165]:
df_30s = df.groupby(['State Code'])[['Prem Single 30 ','Prem Coup 30 ','Coup1 30 ']].median()

#for each row, calculate the decade over decade increase in premiums 

percent_increase_30s = df_30s.pct_change(axis='columns')
percent_increase_30s

,Prem Single 30,Prem Coup 30,Coup1 30
State Code,,,
AK,NaN,1.0,0.337004
AL,NaN,1.0,0.279730
AR,NaN,1.0,0.337004
AZ,NaN,1.0,0.337000
DE,NaN,1.0,0.337008
FL,NaN,1.0,0.337005
HI,NaN,1.0,0.337002
IA,NaN,1.0,0.337004
IL,NaN,1.0,0.337006


In [167]:
df_40s = df.groupby(['State Code'])[['Prem Single 40 ','Prem Coup 40 ','Coup1 40 ']].median()

#for each row, calculate the decade over decade increase in premiums 

percent_increase_40s = df_40s.pct_change(axis='columns')
percent_increase_40s

,Prem Single 40,Prem Coup 40,Coup1 40
State Code,,,
AK,NaN,1.0,0.299293
AL,NaN,1.0,0.248433
AR,NaN,1.0,0.299295
AZ,NaN,1.0,0.299296
DE,NaN,1.0,0.299301
FL,NaN,1.0,0.299294
HI,NaN,1.0,0.299295
IA,NaN,1.0,0.299297
IL,NaN,1.0,0.299297


In [169]:
def scan_pdf(url, word):
    """ scans a pdf from a url looking for a specific word """
    try:
        response = requests.get(url)
        response.raise_for_status()
        pdf_file = io.BytesIP(response.content)
        pdf_reader = PdfReader(pdf_file)

        for page in pdf_reader.pages:
            text = page.extract_text()
            if word in text:
                return True
            return False
    except requests.exceptions.RequestException as e:
        print(f"Error downloading PDF: {e}")
        return False
    except Exception as e:
        print(f"an error occurred: {e}")
        return False

In [191]:
test_url = df['Plan Brochure URL'][200]
search_word = "maternity"

if scan_pdf(test_url, search_word):
    print(f"The word '{search_word}' was found in the PDF.")
else:
    print(f"The word '{search_word}' was not found in the PDF.")

Error downloading PDF: 403 Client Error: Forbidden for url: https://www.premera.com/documents/059782_2025.pdf
The word 'maternity' was not found in the PDF.
